# Final Project
CS696 Big Data

Professor Whitney

Team:

Kristi Werry - 823386935

William Ritchie - 815829203

## Description

## Imports

In [1]:
import numpy as np
import pandas as pd
import sys
from pyspark.sql import Row
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
from pyspark.sql.types import BooleanType
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import StringType
from pyspark.sql.types import DateType
from pyspark.sql.types import TimestampType
from pyspark.sql.functions import to_timestamp, max, length, dayofweek
from pyspark.sql.functions import isnan, when, count, col, max
from pyspark.sql.functions import lit, avg, format_number, corr, udf
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler
from wordcloud import WordCloud, STOPWORDS 
import matplotlib.pyplot as plt

## Functions

In [2]:
def Load_Datasets(sqlContext, files, schema):
    data = sqlContext.createDataFrame([], schema=schema)
    for file in files:
        temp_data = sqlContext.read.csv(path=file[0], schema=schema, dateFormat="yy.dd.MM", timestampFormat="yyyy-MM-dd")
        temp_data = temp_data.withColumn("country", lit(file[1]))
        data = data.union(temp_data)
    return data



def keyWords(df, column, amount):
    temp = Counter(" ".join(df[column].str.lower()).split()).most_common(amount)
    pd.DataFrame(temp, columns=['word', 'count']).plot(kind='bar', x='word', y='count')

    
    
# takes in a dataframe, a grouping, and a column name and returns 
# the average of that column based on the group
def average_by_group(df, group, column):
    return df.groupBy(group).agg(avg(col(column)).alias("average"))



# All credit for this function goes to this site: https://www.geeksforgeeks.org/generating-word-cloud-python/
##takes a long ass time for descriptions ***********~40 mins*************
# iterate through the column
def wordCloud(df, column):
    comment_words = ''
    for val in df[column]: 
      
        # typecaste each val to string 
        val = str(val) 
  
        # split the value 
        tokens = val.split() 
      
        # Converts each token into lowercase 
        for i in range(len(tokens)): 
            tokens[i] = tokens[i].lower() 
      
        comment_words += " ".join(tokens)+" "
  
    wordcloud = WordCloud(width = 800, height = 800, 
                    background_color ='black', 
                    stopwords = stopwords, 
                    min_font_size = 10).generate(comment_words)

    # plot the WordCloud image                        
    plt.figure(figsize = (8, 8), facecolor = None) 
    plt.imshow(wordcloud) 
    plt.axis("off") 
    plt.tight_layout(pad = 0) 
  
    plt.show()
    
    

def keyWords_filtered(df, column, amount):
    pattern = '|'.join(['\|', ';', '-', '\(', '\)', '\[', '\]', '&', ' ', '\.', '\,', '\:', '\/', '\'', '\!', '\$', '’', '\\\\'])
    some[column] = df[column].str.lower().str.replace(pattern, ' ').str.replace(r'\b(\w{1,3})\b', '')
    temp = Counter(" ".join(some[column]).split()).most_common(20)
    pd.DataFrame(temp, columns=['word', 'count']).plot(kind='bar', x='word', y='count')
    

def convert_weekday(num):
    if num == 1:
        return "Monday"
    elif num ==2:
        return "Tuesday"
    elif num ==3:
        return "Wednesday"
    elif num ==4:
        return "Thursday"
    elif num ==5:
        return "Friday"
    elif num ==6:
        return "Saturday"
    elif num ==7:
        return "Sunday"
    else:
        return ""

## Importing Datasets

In [ ]:
sqlContext = SparkSession.builder.appName("FinalProjectYoutube").getOrCreate();
root_dir = "youtube-new/";

# Set up the schema for reading in the data sets into a dataframe
customSchema = StructType([
  StructField("video_id", StringType(), True),
  StructField("trending_date", DateType(), True),
  StructField("title", StringType(), True),
  StructField("channel_title", StringType(), True),
  StructField("category_id", StringType(), True),
  StructField("publish_time", TimestampType(), True),
  StructField("tags", StringType(), True),
  StructField("views", IntegerType(), True),
  StructField("likes", IntegerType(), True),
  StructField("dislikes", IntegerType(), True),
  StructField("comment_count", IntegerType(), True),
  StructField("thumbnail_link", StringType(), True),
  StructField("comments_disabled", BooleanType(), True),
  StructField("ratings_disabled", BooleanType(), True),
  StructField("video_error_or_removed", BooleanType(), True),
  StructField("description", StringType(), True),
  StructField("country", StringType(), True)
])

# Associate csv files with respective countries
data_files = [
   (root_dir + "CAvideos.csv", "Canada"),
   (root_dir + "DEvideos.csv", "Germany"),
   (root_dir + "FRvideos.csv", "France"),
   (root_dir + "GBvideos.csv", "England"),
   (root_dir + "INvideos.csv", "India"),
   (root_dir + "JPvideos.csv", "Japan"),
   (root_dir + "KRvideos.csv", "South Korea"),
   (root_dir + "MXvideos.csv", "Mexico"),
   (root_dir + "RUvideos.csv", "Russia"),
   (root_dir + "USvideos.csv", "US"),
]

# Read in datasets
youtube_data_df = Load_Datasets(sqlContext, data_files, customSchema)

## Dataset Cleaning 

#### Duplicate Rows

In [ ]:
# When dropping the duplicate rows based on video_id we found that half of the dataset gets dropped.  So we now look at the 
# duplicate rows to find out more information about what is going on.  Looking at the duplicate rows you can see that the 
# same video can be trending for mulitple days and in different countries causing the same video to exist in multiple rows.
# We decided the "duplicates" were not actually truely duplicate rows, the information provided by these multiple entries 
# is still useful.  We determined a truely duplicate row requires the same: "video_id", "views", "likes", "dislikes",
# "country", and "trending_date" column values.
pandasdf = pd.DataFrame(data=youtube_data_df.take(100000), columns=youtube_data_df.columns)

pandas_df = pandasdf.loc[pandasdf['video_id'].duplicated()]

# We realized that a number of the duplicates have a "\n" for the video id, we decided to filter those rows out since
# they contain no useful information.  We remove these rows later on when dropping NA values from the dataset.
pandas_df = pandas_df[pandas_df['video_id'] != "\\n"]

# The video id value was manually selected by viewing the resulting dataframe from the previous line. You can see
# that this video was trending for multiple days and in mulitple countries, hence why it has mulitple rows in the 
# the dataset.
pandasdf[pandasdf['video_id'] == "n1WpP7iowLc"]

In [ ]:
# The previous section determined that video_id was not sufficient in determining a truly duplicate row.  The following
# are a combination of the columns we think determine a truly duplicate row.  Meaning, if two rows have the same value
# in all of the below columns, then those two rows are indeed duplicates. 

# We found that if we included views, likes,or dislikes as part of the criteria, then some duplicate rows still existed
# because their view count differed only by roughly 10 views, but everyting else remained the same, which proves
# problematic in future analysis
compare_duplicate_cols = ["video_id", "country", "trending_date"]
row_count_with_dup = youtube_data_df.count()

# Drop duplicate rows, in the event where there is a duplicate row, we want to keep the one with the greatest number of
# views, so we sort the rows by their views column in descending order.  This is useful because the dropDuplicates
# function keeps the first instance of any pair of duplicate rows.
youtube_data_df = youtube_data_df.orderBy(col("views").desc()).dropDuplicates(compare_duplicate_cols)

# View duplicate row count information
num_duplicates = row_count_with_dup - youtube_data_df.count()
print("Number of duplicates: " + str(num_duplicates))
print("Remaining number of rows: " + str(youtube_data_df.count()))

#### NA / Null Values

In [ ]:
# The following are a list of the columns that we determined should not contain a null or NA value.  We could have
# done this when specifying the schema when we were importing the data, but we felt it necessary to learn more information
# about columns that contain nulls and NAs.  After playing around with the data we found that many of the values in the
# description column were NA.  We decided that this was okay because some videos might not have a description, thus
# this is why description is not included in the below list
no_nan_cols = ["video_id", "trending_date", "title", 
        "channel_title", "category_id",  
        "tags", "views", "likes", "dislikes", "comment_count", 
        "thumbnail_link", "comments_disabled", "ratings_disabled", 
        "video_error_or_removed", "country"]

row_count_with_nans = youtube_data_df.count()

# Drop nans from these columns
youtube_data_df = youtube_data_df.na.drop(subset=no_nan_cols) 

# View duplicate row count information
num_nans = row_count_with_nans - youtube_data_df.count()
print("Number of null,nans, and na's: " + str(num_nans))
print("Remaining number of rows: " + str(youtube_data_df.count()))

In [ ]:
# We decided that it was advantageous to replace all of the nulls in the description column with empty strings, this
# way we do not need to check for nulls later on when working with this column in the dataset
youtube_data_df = youtube_data_df.fillna("", subset="description")

# Check that the description column contains no nulls
youtube_data_df.where(youtube_data_df.description.isNull()).count()

#### Filter out #NAME? and #VALUE! from video_id

In [ ]:
# In later analysis we discovered that some rows contained either #NAME? or #VALUE! in their video_id column, which
# we decided was unaccaptable due to questioning the validity of the data in that row.
youtube_data_df = youtube_data_df.filter(youtube_data_df.video_id != "#NAME?")
youtube_data_df = youtube_data_df.filter(youtube_data_df.video_id != "#VALUE!")

#### Dropping duplicates but keeping max views

In [ ]:
# The following filters out the duplicate video entries, leaving only one entry, the one with the highest view count.
# Having a separate dataframe setup this way proves convenient for some future analysis.

# We decided to drop duplicate rows with the same video id and country, and keep the ones with the largest view count
# The orderBy function call and the desc function call reorder the dataset rows in descending order based on the the
# value in the views column.  We wanted this because dropDuplicates function keeps the first instance of any duplicate
# pair, meaning in this case only the duplicate row with the highest view count will remain.
youtube_nodup_df = youtube_data_df.orderBy(col("views").desc()).dropDuplicates(["video_id", "country"])

### Video Averages 

In [ ]:
#Out of the trending videos, average number of views
#change to not duplicate data instead of all_data
average_by_group(youtube_nodup_df, "country", "views").toPandas().plot(kind='bar', y='average', x='country')

In [ ]:
average_by_group(youtube_nodup_df, "country", "likes").toPandas().plot(kind='bar', y='average', x='country')

In [ ]:
average_by_group(youtube_nodup_df, "country", "dislikes").toPandas().plot(kind='bar', y='average', x='country')

In [ ]:
average_by_group(youtube_nodup_df, "country", "comment_count").toPandas().plot(kind='bar', y='average', x='country')

## Finding correlation

In [ ]:
col = ["views", 'likes', 'dislikes', 'comment_count']
vector_col = "corr_features"

inputdf = sqlContext.createDataFrame(youtube_nodup_df.select(*col).collect(), col)

# convert to vector column first
assembler = VectorAssembler(inputCols=inputdf.columns, outputCol=vector_col)
df_vector = assembler.transform(inputdf).select(vector_col)

# get correlation matrix
matrix = Correlation.corr(df_vector, vector_col)
print(str(matrix.head()[0]))

## finding average title and description length

In [ ]:
titlelen = nodup_data.withColumn('titleLength', length('title'))
average_by_group(titlelen, 'country', 'titleLength').toPandas().plot(kind='bar', y='average', x='country')

In [ ]:
desclen = nodup_data.withColumn('descriptionLength', length('description'))
average_by_group(desclen, 'country', 'descriptionLength').toPandas().plot(kind='bar', y='average', x='country')

## possible word cloud
pip install wordcloud

In [ ]:
stopwords = set(STOPWORDS)

In [ ]:
test = us_data.toPandas()
test = test[test['title'].notna()]
test = test[test['description'].notna()]
wordCloud(test, 'title')

In [ ]:
#this is gunna take a very long time!!!!!!!!!!!!!!!!!!!!!!1
#wordCloud(test, 'description')

## Keyword frequency count (us data)

In [ ]:
keyWords(test, "title", 20)

In [ ]:
keyWords(test, "description", 20)

In [ ]:
keyWords_filtered(test, "title", 20)

In [ ]:
keyWords_filtered(test, "description", 20)

## Average Video Trending Duration

In [ ]:
# The following counts the frequency a video_id appears in a country, which we are considering to be the trending 
# duration.  This of course does not account for videos that might re-trend in the same country, we attempted to
# solve this problem but it proved too difficult for pyspark
trend_video_duration_df = youtube_data_df.groupBy('video_id', 'country').count()
trend_video_duration_df.show()
average_by_group(trend_video_duration_df, "country", "count").toPandas().plot(kind='bar', y='average', x='country')

In [ ]:
# The following is the average trending duration across all videos in all countries
trend_video_duration_df.agg(avg(col("count"))).show()

## Video Publish Day of Week Frequency

In [ ]:
# This section calculates what days of the week most trending videos are published on

# This allows convert_weekday to be used on a pyspark column
udf_convert_weekday = udf(convert_weekday, StringType())

# Calculate the frequency of videos published for each day of the week
publish_dow_freq = youtube_nodup_df.withColumn('Number Rep Day of Week', dayofweek(youtube_nodup_df.publish_time)).withColumn('Day of Week',udf_convert_weekday(dayofweek(youtube_nodup_df.publish_time))).groupBy("Day of Week", 'Number Rep Day of Week').count().orderBy('Number Rep Day of Week').drop('Number Rep Day of Week')
publish_dow_freq.show()

# Plot the results
ax = publish_dow_freq.toPandas().plot(kind='bar', figsize=[15,10], y = 'count', x = 'Day of Week', title='Number of Videos Published For Each Day of the Week')
ax.set_xlabel("Day of Week")
ax.set_ylabel("Frequency")

## Trending Video Day of Week Frequency

In [ ]:
# This section calculates what days of the week most trending videos are actually trending

# This allows convert_weekday to be used on a pyspark column
udf_convert_weekday = udf(convert_weekday, StringType())

# Calculate the frequency of videos published for each day of the week
publish_dow_freq = youtube_nodup_df.withColumn('Number Rep Day of Week', dayofweek(youtube_nodup_df.trending_date)).withColumn('Day of Week',udf_convert_weekday(dayofweek(youtube_nodup_df.trending_date))).groupBy("Day of Week", 'Number Rep Day of Week').count().orderBy('Number Rep Day of Week').drop('Number Rep Day of Week')
publish_dow_freq.show()

# Plot the results
ax = publish_dow_freq.toPandas().plot(kind='bar', figsize=[15,10], y = 'count', x = 'Day of Week', title='Number of Videos That Started Trending on a Particular Day of the Week')
ax.set_xlabel("Day of Week")
ax.set_ylabel("Frequency")